In [15]:
import pandas as pd 

df = pd.read_csv('./assets/data/romeo_and_juliet.csv')
df
#proc_data = summarize_lines(dataframe)

,Act,Scene,Line,Player,PlayerLine
0,1,0,1,RICHMOND,"Two households, both alike in dignity, / In fa..."
1,1,1,1,SAMPSON,"Gregory, o' my word, we'll not carry coals."
2,1,1,2,GREGORY,"No, for then we should be colliers."
3,1,1,3,SAMPSON,"I mean, an we be in choler, we'll draw."
4,1,1,4,GREGORY,"Ay, while you live, draw your neck out o' the ..."
...,...,...,...,...,...
836,5,3,61,PRINCE,"This letter doth make good the friar's words, ..."
837,5,3,62,CAPULET,"O brother Montague, give me thy hand: / This i..."
838,5,3,63,MONTAGUE,But I can give thee more: / For I will raise h...
839,5,3,64,CAPULET,"As rich shall Romeo's by his lady's lie, / Poo..."


In [21]:
lines_per_act = df.groupby(['Act']).size().reset_index(name='LineCountTotal')
grouped = df.groupby(['Act', 'Player']).size().reset_index(name='LineCount')
merged = pd.merge(grouped, lines_per_act, on='Act')

merged['LinePercent'] = (merged['LineCount'] / merged['LineCountTotal']) * 100
merged.drop('LineCountTotal' , axis = 1 , inplace=True)

merged

,Act,Player,LineCount,LinePercent
0,1,ABRAHAM,5,2.118644
1,1,BENVOLIO,34,14.406780
2,1,CAPULET,14,5.932203
3,1,FIRST CITIZEN,1,0.423729
4,1,FIRST SERVANT,3,1.271186
...,...,...,...,...
66,5,PARIS,5,5.681818
67,5,PRINCE,11,12.500000
68,5,ROMEO,14,15.909091
69,5,SECOND WATCHMAN,1,1.136364


In [22]:
def label_top5(group):
        # Get top 5 players
        top5 = group.nlargest(5, 'LineCount')
        top5_players = top5['Player'].tolist()
        # Label others as 'OTHER'
        group['Player'] = group['Player'].apply(lambda p: p if p in top5_players else 'OTHER')
        return group


labeled = merged.groupby('Act', group_keys=False).apply(label_top5)

  
final = labeled.groupby(['Act', 'Player'], as_index=False).agg({
    'LineCount': 'sum',
    'LinePercent': 'sum'
})

final

,Act,Player,LineCount,LinePercent
0,1,BENVOLIO,34,14.406780
1,1,JULIET,16,6.779661
2,1,NURSE,20,8.474576
3,1,OTHER,96,40.677966
4,1,ROMEO,50,21.186441
5,1,SAMPSON,20,8.474576
6,2,BENVOLIO,16,7.804878
7,2,JULIET,40,19.512195
8,2,MERCUTIO,34,16.585366
9,2,NURSE,32,15.609756


In [ ]:
def summarize_lines(dataframe) : 
    '''
        Sums each player's total of number of lines and  its
        corresponding percentage per act.

        The sum of lines per player per act is in a new
        column named 'PlayerLine'.

        The percentage of lines per player per act is
        in a new column named 'PlayerPercent'

        Args:
            my_df: The pandas dataframe containing the data from the .csv file
        Returns:
            The modified pandas dataframe containing the
            information described above.
    '''
    